In [72]:
import tensorflow as tf
import numpy as np
from tflearn.layers.conv import conv_1d
sess = tf.InteractiveSession()

In [74]:


n_points = 2048
pc_dim = 3

x = tf.placeholder(tf.float32, [None, n_points, pc_dim])
y = tf.placeholder(tf.float32, [None, n_points, pc_dim])
merged_points = tf.concat([x, y], axis=2)

batch_size = tf.shape(x)[0]

In [81]:
alignment_feat = conv_1d(merged_points, nb_filter=n_points, filter_size=1, strides=1)
# tf.nn.softmax(alignment_feat)
align_loc = tf.cast(tf.argmax(alignment_feat, axis=1), tf.int32) # TODO verify axis

row_idx = tf.range(batch_size, dtype=tf.int32)
row_idx = tf.reshape(tf.tile(row_idx, [n_points]), [n_points, -1])
row_idx = tf.transpose(row_idx)

coords = tf.stack([row_idx, align_loc], axis=2)
aligned_y = tf.gather_nd(y, coords)

<tf.Tensor 'Reshape_10:0' shape=(?, 2048, 2048) dtype=float32>

In [78]:
aligned_y

<tf.Tensor 'GatherNd_9:0' shape=(?, 2048, 3) dtype=float32>

In [80]:
loss = tf.nn.l2_loss(x-aligned_y)
step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Conv1D/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_1/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_1/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_2/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_2/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_3/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_3/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_4/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_4/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_5/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_5/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_6/W:0' shape=(1, 1, 6, 2048) dtype=floa

In [69]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init=tf.global_variables_initializer()
sess.run(init)

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Conv1D/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_1/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_1/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_2/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_2/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_3/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_3/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_4/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_4/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_5/W:0' shape=(1, 1, 6, 2048) dtype=float32_ref>", "<tf.Variable 'Conv1D_5/b:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'Conv1D_6/W:0' shape=(1, 1, 6, 2048) dtype=floa

<tf.Tensor 'sub:0' shape=(?, 2048, 3) dtype=float32>

In [40]:
test_batch_size = 2
x_batch = np.random.random((test_batch_size, n_points, pc_dim))
y_batch = np.random.random((test_batch_size, n_points, pc_dim))

x_batch = np.arange(len(x_batch.flatten())).reshape(test_batch_size, n_points, 3)

test_order = np.hstack([[10, 20, 0, 5, 100], np.arange((n_points)-5), np.arange(n_points)])
test_order = test_order.reshape(test_batch_size, -1)

feed_dict = {x:x_batch, y:y_batch, align_loc:test_order}

sess.run(aligned_x, feed_dict=feed_dict)

array([[[  3.00000000e+01,   3.10000000e+01,   3.20000000e+01],
        [  6.00000000e+01,   6.10000000e+01,   6.20000000e+01],
        [  0.00000000e+00,   1.00000000e+00,   2.00000000e+00],
        ..., 
        [  6.12000000e+03,   6.12100000e+03,   6.12200000e+03],
        [  6.12300000e+03,   6.12400000e+03,   6.12500000e+03],
        [  6.12600000e+03,   6.12700000e+03,   6.12800000e+03]],

       [[  6.14400000e+03,   6.14500000e+03,   6.14600000e+03],
        [  6.14700000e+03,   6.14800000e+03,   6.14900000e+03],
        [  6.15000000e+03,   6.15100000e+03,   6.15200000e+03],
        ..., 
        [  1.22790000e+04,   1.22800000e+04,   1.22810000e+04],
        [  1.22820000e+04,   1.22830000e+04,   1.22840000e+04],
        [  1.22850000e+04,   1.22860000e+04,   1.22870000e+04]]], dtype=float32)

In [211]:
def max_response_coords(batch_size, feat_dim, max_index):
    row_idx = tf.range(batch_size, dtype=tf.int32)
    row_idx = tf.reshape(tf.tile(row_idx, [feat_dim]), [feat_dim, -1])
    row_idx = tf.transpose(row_idx)
    # print row_idx.eval()
    col_idx = tf.range(feat_dim, dtype=tf.int32)
    col_idx = tf.reshape(tf.tile(col_idx, [batch_size]), [batch_size, -1])
    coords = tf.transpose(tf.stack([row_idx, col_idx, max_index]))
    # print coords.eval()
    return coords